In [ ]:
#default_exp query
%load_ext autoreload
%autoreload 2

# Queries

> Este módulo executa as queries sql / MongoDB necessárias para baixar os dados do STEL, RADCOM e MOSAICO

In [ ]:
#export
import requests
from decimal import *
from typing import *
from gazpacho import Soup
from rich.progress import track
from pathlib import Path
from unidecode import unidecode
import pandas as pd
import pandas_read_xml as pdx
import pyodbc
import re
import xml.etree.ElementTree as et
from zipfile import ZipFile
import collections
from fastcore.utils import listify
from fastcore.foundation import L
from fastcore.test import *
from anateldb.constants import *
from pyarrow import ArrowInvalid
getcontext().prec = 5

## Otimização dos Tipos de dados
A serem criados dataframes, normalmente a tipo de data é aquele com maior resolução possível, nem sempre isso é necessário, os arquivos de espectro mesmo possuem somente uma casa decimal, portanto um `float16` já é suficiente para armazená-los. As funções a seguir fazem essa otimização

Code below borrowed from https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2

In [ ]:
#export
def optimize_floats(df: pd.DataFrame, exclude = None) -> pd.DataFrame:
    floats = df.select_dtypes(include=["float64"]).columns.tolist()
    floats = [c for c in floats if c not in listify(exclude)]
    df[floats] = df[floats].apply(pd.to_numeric, downcast="float")
    return df


def optimize_ints(df: pd.DataFrame, exclude=None) -> pd.DataFrame:
    ints = df.select_dtypes(include=["int64"]).columns.tolist()
    ints = [c for c in ints if c not in listify(exclude)]
    df[ints] = df[ints].apply(pd.to_numeric, downcast="integer")
    return df


def optimize_objects(df: pd.DataFrame, datetime_features: List[str], exclude=None) -> pd.DataFrame:
    for col in df.select_dtypes(include=["object"]).columns.tolist():
        if col not in datetime_features:
            if col in listify(exclude): continue
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                dtype = "category"
            else:
                dtype = "string"
            df[col] = df[col].astype(dtype)        
        else:
            df[col] = pd.to_datetime(df[col]).dt.date
    return df


def df_optimize(df: pd.DataFrame, datetime_features: List[str] = [], exclude = None):
    return optimize_floats(optimize_ints(optimize_objects(df, datetime_features, exclude), exclude), exclude)

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [ ]:
#export
def connect_db():
    """Conecta ao Banco ANATELBDRO01 e retorna o 'cursor' (iterador) do Banco pronto para fazer iterações"""
    conn = pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=ANATELBDRO01;"
        "Database=SITARWEB;"
        "Trusted_Connection=yes;"
        "MultipleActiveResultSets=True;",
        timeout=TIMEOUT,
    )
    return conn

In [ ]:
#slow
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (RADCOM, STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

## Funções auxiliares de Formatação
As funções a seguir são utilizadas para formatar diversos objetos intermediários utilizados nas funções de leitura e atualização da base de dados e não são chamadas diretamente. 

In [ ]:
#exporti
def row2dict(row):
    """Receives a json row and return the dictionary from it"""
    return {k: v for k, v in row.items()}


def dict2cols(df, reject=()):
    """Recebe um dataframe com dicionários nas células e extrai os dicionários como colunas
    Opcionalmente ignora e exclue as colunas em reject
    """
    for column in df.columns:
        if column in reject:
            df.drop(column, axis=1, inplace=True)
            continue
        if type(df[column].iloc[0]) == collections.OrderedDict:
            try:
                new_df = pd.DataFrame(df[column].apply(row2dict).tolist())
                df = pd.concat([df, new_df], axis=1)
                df.drop(column, axis=1, inplace=True)
            except AttributeError:
                continue
    return df


def parse_plano_basico(row, cols=COL_PB):
    """Receives a json row and filter the column in `cols`"""
    return {k: row[k] for k in cols}


def scrape_dataframe(id_list):
    df = pd.DataFrame()
    for id_ in track(id_list, description="Baixando informações complementares da Web"):
        html = requests.get(ESTACAO.format(id_))
        df = df.append(pd.read_html(Soup(html.text).find("table").html)[0])
    
    df.rename(columns={'NumFistel': 'Fistel',
                       'Num Serviço': 'Num_Serviço'}, inplace=True)
    return df[["Status", "Entidade", "Fistel", "Frequência", "Classe", 'Num_Serviço', 'Município', 'UF']]

In [ ]:
#exporti
def clean_merge(pasta, df):
    df = df.copy()
    COLS = [c for c in df.columns if "_x" in c]
    for col in COLS:
        col_x = col
        col_y = col.split("_")[0] + "_y"
        if df[col_x].count() > df[col_y].count():
            a, b = col_x, col_y
        else:
            a, b = col_y, col_x

        df.loc[df[a].isna(), a] = df.loc[df[a].isna(), b]
        df.drop(b, axis=1, inplace=True)
        df.rename({a: a[:-2]}, axis=1, inplace=True)

    df.loc[df.Latitude_Transmissor == "", "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor == "", "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor == "", "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor == "", "Longitude_Estação"
    ]
    df.loc[df.Latitude_Transmissor.isna(), "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor.isna(), "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor.isna(), "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor.isna(), "Longitude_Estação"
    ]
    df.drop(["Latitude_Estação", "Longitude_Estação"], axis=1, inplace=True)
    df.rename(
        columns={
            "Latitude_Transmissor": "Latitude",
            "Longitude_Transmissor": "Longitude",
        },
        inplace=True,
    )
    municipios = Path(f"{pasta}/municípios.fth")
    if not municipios.exists():
        municipios = Path(f"{pasta}/municípios.xlsx")
        if not municipios.exists():
            raise FileNotFoundError(f"É necessario a tabela de municípios municípios.fth | municípios.xlsx na pasta {pasta}")
        m = pd.read_excel(municipios, engine='openpyxl')
    else:
        m = pd.read_feather(municipios)
    m.loc[
        m.Município == "Sant'Ana do Livramento", "Município"
    ] = "Santana do Livramento"
    m["Município"] = (
        m.Município.apply(unidecode).str.lower().str.replace("'", " ")
    )  # (lambda x: "".join(e for e in x if e.isalnum()))
    m["UF"] = m.UF.str.lower()
    df["Coordenadas_do_Município"] = False
    df["Latitude"] = df.Latitude.str.replace(",", ".")
    df["Longitude"] = df.Longitude.str.replace(",", ".")
    df["Frequência"] = df.Frequência.str.replace(",", ".")
    df.loc[df["Município"] == "Poxoréo", "Município"] = "Poxoréu"
    df.loc[df["Município"] == "Couto de Magalhães", "Município"] = "Couto Magalhães"
    for row in df[(df.Latitude == "") | (df.Latitude.isna())].itertuples():
        try:
            left = unidecode(row.Município).lower()
            m_coord = (
                m.loc[
                    (m.Município == left) & (m.UF == row.UF.lower()),
                    ["Latitude", "Longitude"],
                ]
                .values.flatten()
                .tolist()
            )
            df.loc[row.Index, "Latitude"] = m_coord[0]
            df.loc[row.Index, "Longitude"] = m_coord[1]
            df.loc[row.Index, "Coordenadas_do_Município"] = True
        except ValueError:
            print(left, row.UF, m_coord)
            continue

    freq_nans = df[df.Frequência.isna()].Id.tolist()
    complement_df = scrape_dataframe(freq_nans)
    df.loc[
        df.Frequência.isna(), ["Status", "Entidade", "Fistel", "Frequência", "Classe", 
                               'Num_Serviço', 'Município', 'UF']
        ] = complement_df.values
        
    for r in df[(df.Entidade.isna()) | (df.Entidade == '')].itertuples():
        df.loc[r.Index, 'Entidade'] = ENTIDADES.get(r.Fistel, '')

    df.loc[df["Número_da_Estação"] == "", "Número_da_Estação"] = -1
    df["Latitude"] = df["Latitude"].astype("float")
    df["Longitude"] = df["Longitude"].astype("float")
    df["Frequência"] = df.Frequência.astype("float")
    df.loc[df.Serviço == 'OM', 'Frequência'] = df.loc[df.Serviço == 'OM', 'Frequência'].apply(lambda x: Decimal(x) / Decimal(1000))
    df["Frequência"] = df.Frequência.astype("float")
    df['Validade_RF'] = df.Validade_RF.astype('string').str.slice(0,10)
    df.loc[df['Num_Ato'] == '', 'Num_Ato'] = -1
    df['Num_Ato'] = df.Num_Ato.astype('int')
    return df_optimize(df, exclude=['Latitude', 'Longitude', 'Frequência'])

In [ ]:
#exporti
def read_estações(path):
    def extrair_ato(row):
        if not isinstance(row, str):
            row = listify(row)[::-1]
            for d in row:
                if not isinstance(d, dict):
                    continue
                if (d.get("@TipoDocumento") == "Ato") and (
                    d.get("@Razao") == "Autoriza o Uso de Radiofrequência"
                ):
                    return d["@NumDocumento"], d["@DataDOU"][:10]
            else:
                return "", ""
        return "", ""

    es = pdx.read_xml(path, ["estacao_rd"])
    dfs = []
    for i in range(es.shape[0]):
        df = pd.DataFrame(es["row"][i]).replace("", pd.NA)
        df = dict2cols(df)
        df.columns = [unidecode(c).lower().replace("@", "") for c in df.columns]
        dfs.append(df)
    df = pd.concat(dfs)
    df = df[df.state.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    docs = L(df.historico_documentos.apply(extrair_ato).tolist())
    return df
    df = df.loc[:, COL_ESTACOES]
    df["Num_Ato"] = docs.itemgot(0).map(str)
    df["Data_Ato"] = docs.itemgot(1).map(str)
    df.columns = NEW_ESTACOES
    df['Validade_RF'] = df.Validade_RF.astype('str').str.slice(0,10)
    df["Data_Ato"] = df.Data_Ato.str.slice(0,10)
    df['Entidade'] = df.Entidade.fillna('')
    ENTIDADES.update({r.Fistel : r.Entidade for r in df.itertuples() if r.Entidade != ''})
    return df


def read_plano_basico(path):
    pb = pdx.read_xml(path, ["plano_basico"])
    # df = pd.DataFrame(df["row"].apply(row2dict).tolist()).replace("", pd.NA)
    dfs = []
    for i in range(pb.shape[0]):
        df = pd.DataFrame(pb["row"][i]).replace("", pd.NA)
        df = dict2cols(df)
        df.columns = [unidecode(c).lower().replace("@", "") for c in df.columns]
        dfs.append(df)
    df = pd.concat(dfs)
    df = df.loc[df.pais == "BRA", COL_PB].reset_index(drop=True)
    df.columns = NEW_PB
    df.sort_values(["Id", "Canal"], inplace=True)
    df['Entidade'] = df.Entidade.fillna('')
    ENTIDADES.update({r.Fistel : r.Entidade for r in df.itertuples() if r.Entidade != ''})
    df = df.groupby("Id", as_index=False).first()  # remove duplicated with NaNs
    df.dropna(subset=['Status'], inplace=True)
    df = df[df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(drop=True)
    return df

def read_historico(path):
    regex = r"\s([a-zA-Z]+)=\'{1}([\w\-\ :\.]*)\'{1}"
    with ZipFile(path) as xmlzip:
        with xmlzip.open("documento_historicos.xml", "r") as xml:
            xml_list = xml.read().decode().split("\n")[2:-1]
    dict_list = []
    for item in xml_list:
        matches = re.finditer(regex, item, re.MULTILINE)
        dict_list.append(dict(match.groups() for match in matches))
    df = pd.DataFrame(dict_list)
    df = df[
        (df.tipoDocumento == "Ato") & (df.razao == "Autoriza o Uso de Radiofrequência")
    ].reset_index()
    df = df.loc[:, ["id", "numeroDocumento", "orgao", "dataDocumento"]]
    df.columns = ["Id", "Num_Ato", "Órgao", "Data_Ato"]
    df["Data_Ato"] = pd.to_datetime(df.Data_Ato)
    return df.sort_values("Data_Ato").groupby("Id").last().reset_index()


## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [ ]:
#export
def update_radcom(pasta):
    """Atualiza a tabela local retornada pela query `RADCOM`"""
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="earth"
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(RADCOM, conn)
            df = df_optimize(df, exclude=['Frequência'])
            try:
                df.to_feather(f"{pasta}/radcom.fth")
            except ArrowInvalid:
                df.to_excel(f"{pasta}/radcom.xlsx", engine='openpyxl', index=False)
        except pyodbc.OperationalError:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )


def update_stel(pasta):
    """Atualiza a tabela local retornada pela query `STEL`"""
    with console.status(
        "[magenta]Lendo o Banco de Dados do STEL. Processo Lento, aguarde...",
        spinner="moon",
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(STEL, conn)
            df['Validade_RF'] = df.Validade_RF.astype('str').str.slice(0,10)
            df = df_optimize(df, exclude=['Frequência'])
            try:
                df.to_feather(f"{pasta}/stel.fth")
            except ArrowInvalid:
                df.to_excel(f"{pasta}/stel.xlsx", engine='openpyxl', index=False)
        except pyodbc.OperationalError:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )


def update_mosaico(pasta):
    """Atualiza a tabela local do Mosaico. É baixado e processado arquivos xml zipados da página pública do Spectrum E"""
    with console.status(
        "[blue]Baixando as Estações do Mosaico...", spinner="shark"
    ) as status:
        file = requests.get(ESTACOES)
        with open(f"{pasta}/estações.zip", "wb") as estações:
            estações.write(file.content)
    with console.status(
        "[blue]Baixando o Plano Básico das Estações...", spinner="weather"
    ) as status:
        file = requests.get(PLANO_BASICO)
        with open(f"{pasta}/Canais.zip", "wb") as plano_basico:
            plano_basico.write(file.content)
    console.print(":package: [blue]Consolidando as bases de dados...")
    estações = read_estações(f"{pasta}/estações.zip")
    plano_basico = read_plano_basico(f"{pasta}/Canais.zip")
    df = estações.merge(plano_basico, on="Id", how="left")
    df = clean_merge(pasta, df)
    try:
        df.reset_index(drop=True).to_feather(f"{pasta}/mosaico.fth")
    except ArrowInvalid:
        with pd.ExcelWriter(f"{pasta}/mosaico.xlsx") as workbook:
            df.reset_index(drop=True).to_excel(workbook, sheet_name='Sheet1', engine="openpyxl", index=False)
    console.print("Kbô :vampire:")
    return df


def update_base(pasta, up_stel=False, up_radcom=False, up_mosaico=False):
    """Wrapper que atualiza opcionalmente lê e atualiza as três bases indicadas anteriormente, as combina e salva o arquivo consolidado na pasta `pasta`"""
    stel = read_stel(pasta, up_stel).loc[:, TELECOM]
    stel.rename(
        columns={"Serviço": "Num_Serviço", "Número da Estação": "Número_da_Estação"},
        inplace=True,
    )
    radcom = read_radcom(pasta, up_radcom)
    radcom.rename(columns={"Número da Estação": "Número_da_Estação"}, inplace=True)
    mosaico = read_mosaico(pasta, up_mosaico)
    radcom["Num_Serviço"] = 231
    radcom["Status"] = "RADCOM"
    radcom["Classe"] = radcom.Fase.str.strip() + "-" + radcom.Situação.str.strip()
    radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
    radcom["Num_Ato"] = -1
    radcom["Data_Ato"] = ""
    radcom["Validade_RF"] = ""
    radcom = radcom.loc[:, RADIODIFUSAO]
    stel["Num_Ato"] = -1
    stel["Data_Ato"] = ""
    stel['Entidade'] = stel.Entidade.str.rstrip().str.lstrip()
    mosaico = mosaico.loc[:, RADIODIFUSAO]
    rd = mosaico.append(radcom)
    rd = rd.append(stel).sort_values("Frequência").reset_index(drop=True)
    rd = df_optimize(rd, exclude=['Frequência'])
    try:
        rd.to_feather(f"{pasta}/base.fth")
    except ArrowInvalid:
        with pd.ExcelWriter(f"{pasta}/base.xlsx") as workbook:
            rd.to_excel(workbook, sheet_name='Sheet1', engine="openpyxl", index=False)
    return rd


## Funções de Leitura das diversas bases
A presente biblioteca utiliza três bases de dados: 
* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão comunitária
* `MOSAICO` - Demais serviços de Radiodifusão e paulatinamente também adicionado serviços privados

As funções a seguir são para leitura dos arquivos locais baixados e processados dessas bases, opcionalmente esses arquivos podem ser atualizados antes de serem lidos passando o argumento `update = True`

In [ ]:
#export
def read_stel(pasta, update=False):
    """Lê o banco de dados salvo localmente do STEL. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01 caso `update = True` ou não exista o arquivo local"""
    if update:
        update_stel(pasta)
    file = Path(f"{pasta}/stel.fth")
    try:
        stel = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/stel.xlsx")
        try:
            stel = pd.read_excel(file, engine="openpyxl")
        except FileNotFoundError:
            read_stel(pasta, True)
    return stel


def read_radcom(pasta, update=False):
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01 caso `update = True` ou não exista o arquivo local"""
    if update:
        update_radcom(pasta)
    file = Path(f"{pasta}/radcom.fth")
    try:
        radcom = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/radcom.xlsx")
        try:
            radcom = pd.read_excel(file, engine="openpyxl")
        except FileNotFoundError:
            read_radcom(pasta, True)
    return radcom


def read_mosaico(pasta, update=False):
    """Lê o banco de dados salvo localmente do MOSAICO. Opcionalmente o atualiza antes da leitura baixando os diversos arquivos disponíveis na interface web pública"""
    if update:
        update_mosaico(pasta)
    file = Path(f"{pasta}/mosaico.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/mosaico.xlsx")
        try:
            df = pd.read_excel(file)
        except FileNotFoundError:
            return read_mosaico(pasta, update=True)
    return df_optimize(df, exclude=['Frequência'])
    

def read_base(pasta, up_stel=False, up_radcom=False, up_mosaico=False):
    """Wrapper que combina a chamada das três funções de leitura do banco e opcionalmente é possível atualizá-las antes da leitura"""
    if any([up_stel, up_radcom, up_mosaico]):
        update_base(pasta, up_stel, up_radcom, up_mosaico)
    file = Path(f"{pasta}/base.fth")
    try:
        df =  pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/base.xlsx")
        try:
            df = pd.read_excel(file, engine='openpyxl')
        except FileNotFoundError:
            df = update_base(pasta, up_stel, up_radcom, up_mosaico)
    return df_optimize(df, exclude=['Frequência'])

In [ ]:
pasta = r'G:\Meu Drive\repos\Code\BaseDados'

In [ ]:
stel = read_stel(pasta, True)

🌕  Lendo o Banco de Dados do STEL. Processo Lento, aguarde...

In [ ]:
stel.tail()

In [ ]:
radcom = read_radcom(pasta, True)

In [ ]:
radcom.tail()

In [ ]:
mosaico = read_mosaico(pasta)

In [ ]:
mosaico.tail()

In [ ]:
base = read_base(pasta)

In [ ]:
base

In [ ]:
base.sample(10)

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato
104257,157.73125,19,NaN,NaN,RUMO MALHA CENTRAL S.A.,50418927251,1010234894,Santa Isabel,GO,-15.324844,-49.376007,2040-02-07,-1,NaN
369472,459.83750,19,NaN,NaN,LIDERSUL SEGURANCA PRIVADA EIRELI,50416383734,1006689653,São José dos Pinhais,PR,-25.544958,-49.206837,2038-05-15,-1,NaN
253982,168.23125,19,NaN,NaN,POLICIA MILITAR DO ESTADO DE MINAS GERAIS,50401288943,1004062220,Santa Luzia,MG,-19.796352,-43.919338,2037-06-16,-1,NaN
405362,932.56250,19,NaN,NaN,CEMIG DISTRIBUICAO S.A,50402659058,683549049,Ribeirão das Neves,MG,-19.747778,-44.146946,2030-11-08,-1,NaN
230015,167.85625,19,NaN,NaN,POLICIA MILITAR DO ESTADO DE MINAS GERAIS,50401288943,1007982516,Conceição do Mato Dentro,MG,-19.034672,-43.423973,2037-06-16,-1,NaN
316436,383.22500,19,NaN,NaN,CONCESSIONARIA DO SISTEMA ANHANGUERA-BANDEIRAN...,50001442015,430890052,Jundiaí,SP,-23.242525,-46.902634,2038-08-17,-1,NaN
21242,148.03000,19,NaN,NaN,AGRICOLA MORENO DE NIPOÃ LTDA,50411773844,695281321,Monte Aprazível,SP,-20.687666,-49.689304,2041-04-15,-1,NaN
309160,368.58750,19,NaN,NaN,GERDAU AÇOMINAS S/A,4030000843,698419316,Congonhas,MG,-20.543436,-43.741840,2023-07-16,-1,NaN
85198,156.57500,604,NaN,NaN,IPIRANGA PRODUTOS DE PETROLEO S.A.,50415298091,1010898679,Itaituba,PA,-4.294445,-56.018890,2037-10-06,-1,NaN
22515,148.11000,19,NaN,NaN,SAO MARTINHO S/A,2030099406,522969917,Iracemápolis,SP,-22.585897,-47.531334,2027-02-12,-1,NaN


In [ ]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433838 entries, 0 to 433837
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Frequência         433838 non-null  float64 
 1   Num_Serviço        433838 non-null  int16   
 2   Status             28147 non-null   category
 3   Classe             28122 non-null   category
 4   Entidade           433827 non-null  category
 5   Fistel             433838 non-null  int64   
 6   Número_da_Estação  433838 non-null  int32   
 7   Município          433838 non-null  category
 8   UF                 433838 non-null  category
 9   Latitude           433838 non-null  float32 
 10  Longitude          433838 non-null  float32 
 11  Validade_RF        427003 non-null  category
 12  Num_Ato            433838 non-null  int32   
 13  Data_Ato           13171 non-null   category
dtypes: category(7), float32(2), float64(1), int16(1), int32(2), int64(1)
memory usage: 1

In [ ]:
base.to_feather(f'{pasta}/base.fth')

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted constants.ipynb.
Converted filter.ipynb.
Converted index.ipynb.
Converted queries.ipynb.
